<font size="1"> *This notebook is best viewed in jupyter lab/notebook. You may also choose to use Google Colab but some parts of the images/colouring will not be rendered properly.</font> 

<div class="alert alert-block alert-warning">
    
# Assignment

Welcome to task 4 of the assignment! This notebook contains all the code and comments that you need to submit. The places where you need to edit are highlighted in red. Feel free to add in your own markdown for additional comments.

__Submission details: make sure you all your outputs have been generated. Submit this Jupyter Notebook.__

<b>Enter you student details below</b>

- <b>Student Name:</b> Firstname Lastname
- <b>Student ID:</b> 123456789  

<div class="alert alert-block alert-danger">

## Table of Contents

* [Libraries](#Libraries)
    
* [Task 4.1: Create dataloaders and visualize the dataset](#t5_1)
    
* [Task 4.2: Load and Apply Transfer Learning with Pre-trained Model](#t5_2)
    
* [Task 4.3: Train and Test the model](#t5_3)
    
* [Task 4.4: Visualise predicted landmarks for some of the images](#t5_4)
    
* [Task 4.5: Discussion Questions](#t5_5)

<div class="alert alert-block alert-warning">

# Libraries

Libraries are important as it saves us time from writing our own functions all the time such as graphing, or creating matrices. Brief library descriptions have been added for every library that we import. You may also check the official documentations for more details.

The required libraries are specified in this notebook for you.

In [ ]:
# Mount Google Drive
import os
try:
    from google.colab import drive
    colabFlag = True
    
    PATH_DATASETS = '/content/gdrive/MyDrive/...' # Add in your own path here to the folder containing the data
    # mount Google Drive
    drive.mount('/content/gdrive')
    print(f"Using Google Colab with file path {PATH_DATASETS}")
except:
    colabFlag = False

    PATH_DATASETS = os.environ.get("PATH_DATASETS", "./data")

    print(f"Not using Google Colab. File path will be: {PATH_DATASETS}")

In [ ]:
# If you run on google colab uncomment bellow comment
# !pip install torchmetrics pytorch-lightning torchvision -qq
# !pip install imutils

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data.dataset import Dataset
from torchmetrics import MeanSquaredError

import pytorch_lightning as pl ## Pytorch lightning is a wrapper for pytorch that makes it easier to train models
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar

# Setting seeds for reproducibility
pl.seed_everything(4179)
random.seed(4179)
np.random.seed(4179)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


BATCH_SIZE = 64 if torch.cuda.is_available() else 64
# Define GPU number
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')
num_workers = 0 ## Set to your device

<div class="alert alert-block alert-info">

# Face Landmarks Detection with Transfer Learning
    
## What you should do in this task!

In this task you will be applying transfer learning on a ResNet50 model for the Face Landmark Detection task using Pytorch and PyTorch Lightning.

You have to use Pytorch inbuilt models to load the ResNet50 model and its pre-trained parameters, then use the Pytorch Lightning module class to modify the ResNet50 model architecture in order to detect landmarks of human faces. As always, we will train our model using stochastic gradient descent (SGD).

#### In this task, you will work on the following points:
 1. Prepare the dataset and create dataloaders with suggested transformations.
 2. Visualise a training sample with ground truth landmark points.
 3. Modify the ResNet50 model.
 4. Perform training and evaluation of the model.
 5. Visualise your experimental results.
    
<img src="figures/face_landmarks.png" width="700" align="center">

<div class="alert alert-block alert-info">
    
### Task 4.1: Create dataloaders and visualize the dataset <a class="anchor" id="t5_1"></a>
    
In this section you have to create dataloaders for the face landmarks dataset. To do so, begin by crafting a Dataset class called **FLD_Data** to read data from **face_landmark.npz**. Then, visualise a few samples of the data along with the landmarks.

The FLD data can be loaded in completely since it is relatively small.

Have a look at how the data is loaded in to understand how to access the data for the modelling process. You do NOT have to edit the FLD_Data class!

In [3]:
class FLD_Data(Dataset):
    def __init__(self, trn_tst = 0, transform=None):
        # Load numpy data
        data = np.load(PATH_DATASETS + '/face_landmark.npz')
        if trn_tst == 0:
            # Create dataset for trainloader 
            self.images = ???
            self.landmarks = ???
        else:
            # Create dataset for testloader 
            self.images = ???
            self.landmarks = ???
            
        self.transform = ???   ## Save the transforms as your class' attributes

    # Define len function
    def __len__(self):
        return ???

    # Define getitem function
    def __getitem__(self, idx):

        images = ???
        landmarks = ???
        
        if self.transform is not None:  ## If there is a transform, apply it here to your sample
            ???
            
        return images, landmarks ## Useful for your loss later on

In [4]:
# Define a series of image transformations to be applied to data
data_transforms = transforms.Compose([
    ???,     # Convert the image to a PyTorch tensor
    ???,     # Normalize pixel values to have a mean and standard deviation of 0.5
])

In [5]:
## Set your train and test datasets
train_dataset = FLD_Data(trn_tst=0, transform=data_transforms)
test_dataset = FLD_Data(trn_tst=1, transform=data_transforms)

Using the 'random_split' function, create a validation dataset by using 10% of training data.

In [ ]:
# split the dataset into validation and test sets
# use 10% data as validation data from training data
len_valid_set = int(0.1*len(train_dataset)) ## 10% of the data data
len_train_set = len(train_dataset) - len_valid_set  ## 90% of the remaining train data

train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset , [len_train_set, len_valid_set])  ## Split train data

# shuffle relevant datasets and use the torch dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# You can print the length of the train, validation, and test set to ensure they are correct
print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_set))
print("The length of Test set is {}".format(len(test_dataset)))

In [7]:
# Write visualize_dataloader function to visualize 3 images from a given data loader

# Define a function to visualize images from a data loader
# Input parameters: dataloader 
def visualize_dataloader(dataloader):

    # Create a figure with subplots to display images
    ???

    # Get a batch of images and labels from the data loader
    ???

    # Loop through the first 3 images in the batch
    ???

        # Un-normalize the image
        ??? 

        # Display the image with the landmarks (labels) in the subplot
        ???

    # Show the plot
    plt.show()

In [ ]:
# Visualise 3 images as well as their corresponding facial landmarks from the test_loader

visualize_dataloader(test_loader)

<div class="alert alert-block alert-info">
    
### Task 4.2: Load and Apply Transfer Learning with Pre-trained Base Model <a class="anchor" id="t5_2"></a>

Transfer learning can be employed when the goal is to use pre-trained models for a new task. The new task can be classification or numerical estimation, but the important thing about transfer learning is that you need to ensure the dataset used for the pre-training is similar in its distribution to the new task. For example, if you pre-trained model was only trained on classifcation of coloured images, it will not perform well for black and white images! 

In this task you need to:
- Load in a **resnet50** model **WITH** pretrained weights and print model structure of resnet50
- Replace the current FC layer in the **resnet50** model with a new FC layer 


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Load the model using torchvision and print the model structure
model = ???
print(model)

In [10]:
class FaceLandmarkDetection(pl.LightningModule):
    def __init__(self, learning_rate=0.001, num_classes=136):
        super().__init__()
        self.learning_rate = learning_rate 
        self.loss_fun = ???  ## Define your loss! What should it be?
        
        ## Load the pre-trained network as follows
        ## Use the importedweights of:   ResNet50_Weights.IMAGENET1K_V1
        ## See pytorch documentation on how to load models with a pre-trained weight
        ???
        
        ## Freeze the model weights (see PDF's hints)
        ## You can set the param.requires_grad = False 
        ## That way those parameters will not get updated
        ???

        ## Remake the FC layer. You can use the in_features from the original model 
        ???

        ## Which error do we use? Remember, we are dealing with minimising coordinate errors in the landmarks
        ## This should be the same as your loss function
        self.val_error = ???
        self.train_error = ???
        self.test_error = ???
            
    def forward(self, x):
        #Pass input through conv layers
        x_out = ???
    
        return x_out
    
    def training_step(self, batch, batch_idx):
        ## Obtain predictions
        ???
        
        ???  ## Ensure you reshape your landmarks (labels) data to compare against the preds
        loss = ???
        error = ???
        
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("train_error", error, prog_bar=True, on_step=False, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        ???

        loss = ???
        error = ???

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_error", error, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        ???

        loss = ???

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("test_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        x, y = batch
        pred = self(x)
        
        return (pred, y, x)   ## Return 3 values here: predicted landmarks, original landmarks and the original image

    def configure_optimizers(self):
        ???

        return ???

    ####################
    # DATA RELATED HOOKS
    ####################

    def train_dataloader(self):
        return train_loader
    
    def val_dataloader(self):
        return valid_loader

    def test_dataloader(self):
        return test_loader

<div class="alert alert-block alert-info">
    
### Task 4.3: Train and Test the model <a class="anchor" id="t5_3"></a>

In [ ]:
## Initialise your face landmark model
## Train until the end of max_epochs or if the early stopping condition has occured
## Note that if you implement it similarly to how we have done it (same hyper parameters etc.)
## Then your epoch should stop around ~110 (+/- 25 epochs)

FaceLandmarkDetection_model = FaceLandmarkDetection()

# Define checkpoint callback function to save best model
checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath="checkpoints_task_4/",
        save_top_k=1,        # save the best model
        mode="min",
        every_n_epochs=1
    )

# Train and test the model
trainer = pl.Trainer(
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  
    max_epochs=50,
    callbacks=[TQDMProgressBar(refresh_rate=20), checkpoint_callback],
    logger=CSVLogger(save_dir="logs_task_4/"),
)
trainer.fit(FaceLandmarkDetection_model)
trainer.test()

In [ ]:
## Plot your train and validation losses!

# read logs 
metrics_task = pd.read_csv(trainer.logger.log_dir + "/metrics.csv")
metrics_task.set_index("epoch", inplace=True)
metrics_task = metrics_task.groupby(level=0).sum().drop("step", axis=1)

# Plot using matplotlib to plot your results
???

<div class="alert alert-block alert-info">
    
### Task 4.4: Visualise predicted landmarks for some of the images <a class="anchor" id="t5_4"></a>
    
Utilise the trainer predict method to output the predicted landmarks. Then overlay this with the original landmarks and the original image.

In [ ]:
## Scatter plot for model trained using SGD
# Get the images, labels, and predictions
???

# Create a subplot with 1 row and 3 columns to display images
???

# Iterate through the first 3 samples for visualization
??? 

# Show the plotted images
plt.show()

<div class="alert alert-block alert-success">
    
### Task 4.5: Discussion Questions <a class="anchor" id="t5_5"></a>

### Questions
1. What are two things to keep in mind about the data when choosing your pre-trained model? Why are these important? 

3. What was your thought process in finetuning your model to achieve the required test loss?
    
### Answers
    
1. 

2. 
